In [1]:
import numpy as np
import pandas as pd

import os
import glob
import librosa
import tqdm
import pickle

from collections import Counter, defaultdict

from torchvision.datasets.folder import default_loader
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
import torch.nn.functional as F
import torch

DATA_PATH = './data'
join_path = lambda path: os.path.join(DATA_PATH, path)

In [4]:
filepaths = glob.glob(join_path('train_curated/*')) # именно в таком порядке будут идти все аудиозаписи

train = pd.read_csv(join_path('train_curated.csv')).set_index('fname')

cats = list(set(sum(list(train['labels'].str.split(',').values), [])))

train['enc_labels'] = train['labels'].str.split(',').apply(lambda x: [int(label in x) for label in cats])

targets = np.array(train.loc[[fp[-12:] for fp in filepaths], 'enc_labels'].tolist())

In [5]:
with open(join_path('train_curated_mels.pkl'), 'rb') as f:
    data = pickle.load(f)

In [6]:
import mclnn
base = nn.Sequential(
    mclnn.CLNNModule(128, 256, 4, dilation=1, mask_params={"bandwidth":64, "overlap":32}),
    nn.LeakyReLU(),
    mclnn.BatchNorm(256),
    mclnn.CLNNModule(256, 256, 4, dilation=1, mask_params={"bandwidth":64, "overlap":32}),
    nn.LeakyReLU(),
    mclnn.TemporalPool(2),
    mclnn.BatchNorm(256),
    mclnn.CLNNModule(256, 256, 4, dilation=1),
    nn.LeakyReLU(),
    mclnn.TemporalPool(2),
    mclnn.CLNNModule(256, 256, 4),
    nn.LeakyReLU(),
    mclnn.TemporalPool(4),
    mclnn.BatchNorm(256),
    mclnn.Flatten(),
    nn.Linear(256*8, 1024),
    nn.LeakyReLU(),
    nn.Linear(1024, 80)
)

class W(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

        
    def forward(self, input):
        input = input.squeeze(1)
        return self.model(input)
    
model = W(base)

In [7]:
# pip install scikit-multilearn 
from skmultilearn.model_selection import iterative_train_test_split

train_X, train_y, test_X, test_y = iterative_train_test_split(X=data, y=targets, test_size=0.1)

In [8]:
train_ds = TensorDataset(torch.tensor(train_X).unsqueeze(1), torch.tensor(train_y).float())
val_ds = TensorDataset(torch.tensor(test_X).unsqueeze(1), torch.tensor(test_y).float())

train_dl = DataLoader(train_ds, batch_size=64)
val_dl = DataLoader(val_ds, batch_size=64)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = Model(num_classes=80)
logdir = join_path('logdir')
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [10]:
from kirill.flame import Trainer # мой небольшой пайплайн для тренировки сеток

trainer = Trainer(model, optimizer, criterion, device, logdir)

CRITICAL:root:Cannot load caffe2.python. Error: /opt/caffe2/build/caffe2/python/caffe2_pybind11_state.so: undefined symbol: _ZN5fLI6438FLAGS_caffe2_max_keep_on_shrink_memoryE


In [12]:
# # пример запуска для минимизации логлосса
trainer.train(train_dl, val_dl, n_epochs=240, verbose=True)


# losses, best_loss, epoch_time = utils.train_model(trainer, train_dl, val_dl, val_y, scheduler,
#                                             n_epochs=1000, gap=20, verbose=True)



  1 epoch. train: 0.00036. val: 0.00093. time: 8.967s.
  2 epoch. train: 0.00030. val: 0.00090. time: 17.942s.
  3 epoch. train: 0.00025. val: 0.00092. time: 26.956s.
  4 epoch. train: 0.00021. val: 0.00100. time: 35.994s.
  5 epoch. train: 0.00017. val: 0.00104. time: 45.032s.
  6 epoch. train: 0.00014. val: 0.00107. time: 54.098s.
  7 epoch. train: 0.00013. val: 0.00107. time: 63.193s.
  8 epoch. train: 0.00010. val: 0.00107. time: 72.303s.
  9 epoch. train: 0.00009. val: 0.00112. time: 81.403s.
 10 epoch. train: 0.00008. val: 0.00118. time: 90.565s.
 11 epoch. train: 0.00007. val: 0.00118. time: 99.751s.
 12 epoch. train: 0.00006. val: 0.00124. time: 108.925s.
 13 epoch. train: 0.00005. val: 0.00128. time: 118.070s.
 14 epoch. train: 0.00003. val: 0.00132. time: 127.267s.
 15 epoch. train: 0.00002. val: 0.00130. time: 136.095s.
 16 epoch. train: 0.00002. val: 0.00136. time: 145.239s.
 17 epoch. train: 0.00002. val: 0.00141. time: 154.411s.
 18 epoch. train: 0.00001. val: 0.00145. ti

144 epoch. train: 0.00000. val: 0.00225. time: 1314.468s.
145 epoch. train: 0.00000. val: 0.00225. time: 1323.614s.
146 epoch. train: 0.00000. val: 0.00225. time: 1332.790s.
147 epoch. train: 0.00000. val: 0.00226. time: 1341.935s.
148 epoch. train: 0.00000. val: 0.00226. time: 1351.094s.
149 epoch. train: 0.00000. val: 0.00227. time: 1360.251s.
150 epoch. train: 0.00000. val: 0.00227. time: 1369.391s.
151 epoch. train: 0.00000. val: 0.00228. time: 1378.528s.
152 epoch. train: 0.00000. val: 0.00228. time: 1387.550s.
153 epoch. train: 0.00000. val: 0.00229. time: 1396.667s.
154 epoch. train: 0.00000. val: 0.00229. time: 1405.809s.
155 epoch. train: 0.00000. val: 0.00230. time: 1414.941s.
156 epoch. train: 0.00000. val: 0.00230. time: 1424.055s.
157 epoch. train: 0.00000. val: 0.00231. time: 1433.175s.
158 epoch. train: 0.00000. val: 0.00231. time: 1442.339s.
159 epoch. train: 0.00000. val: 0.00232. time: 1451.468s.
160 epoch. train: 0.00000. val: 0.00232. time: 1460.559s.
161 epoch. tra

In [18]:
from metrics import lwlrap_score

# если хочется считать скор метрики оргов
losses = []
for i in range(20):
    # делает один проход по даталоадеру
    trainer.train_step(train_dl, verbose=True)
    
    # формируем предсказания
    model.eval()
    preds = []
    for batch in tqdm.tqdm(val_dl):
        batch = batch[0].to(device)
        pred = model(batch)
        preds.append(pred.cpu().detach().numpy())
    preds = np.vstack(preds)
    
    # считаем скор метрики оргов
    score = lwlrap_score(test_y, preds)
    print(score)
    
    losses.append(score)

  0%|          | 0/70 [00:00<?, ?it/s]

0.32207662069576626


  0%|          | 0/70 [00:00<?, ?it/s]

0.4166380930591757


  0%|          | 0/70 [00:00<?, ?it/s]

...

0.6772803548081768


  0%|          | 0/70 [00:00<?, ?it/s]

0.6778412922729062


100%|██████████| 8/8 [00:00<00:00, 11.81it/s]

0.6585010786758971


In [ ]:
# test inference

with open(join_path('test_mels.pkl'), 'rb') as f:
    test_data = pickle.load(f)
    
test_filepaths = glob.glob(join_path('test/*'))

test_ds = TensorDataset(torch.tensor(test_data).unsqueeze(1))
test_dl = DataLoader(test_ds, batch_size=64)

model.eval()
preds = []
for batch in tqdm.tqdm(test_dl):
    batch = batch[0].to(device)
    pred = model(batch)
    preds.append(pred.cpu().detach().numpy())
preds = np.vstack(preds)

In [ ]:
sub = pd.DataFrame(preds)
sub.columns = [label for label in cats]
sub['fname'] = [fp[-12:] for fp in test_filepaths]
sub.to_csv("sub3.csv", index=False)